In [1]:
import os
os.chdir('..')

In [2]:
#add to entity
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_is_augmented: bool

In [3]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
TRAINING_DATA_PATH

WindowsPath('C:/Users/anasm/OneDrive/Desktop/Image Processing/Kidney disease detection/Data')

In [5]:
#add to configuration
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        param_filepath=PARAM_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.param=read_yaml(param_filepath)

        create_directories(self.config.training.root_dir)



    def get_training_configs(self) ->PrepareTrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model

        training_config=PrepareTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=TRAINING_DATA_PATH,
            params_epochs=self.param.EPOCHS,
            params_is_augmented=self.param.AUGMENTATION
        )
        return training_config

In [6]:
#add to components
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
class Trainer:
    def __init__(self, config: PrepareTrainingConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def load_data(self):
        self.normal_path=os.path.join(self.config.training_data, 'Normal')
        self.tumor_path=os.path.join(self.config.training_data, 'Tumor')
        self.stone_path=os.path.join(self.config.training_data, 'Stone')
        self.cyst_path=os.path.join(self.config.training_data, 'Cyst')

        self.normal_labels=[]#encode to (1, 0, 0, 0)
        self.tumor_labels=[]#encode to (0, 1, 0, 0)
        self.stone_labels=[]#encode to (0, 0, 1, 0)
        self.cyst_labels=[]#encode to (0, 0, 0, 1)

        self.normal_tensors=[]
        self.tumor_tensors=[]
        self.stone_tensors=[]
        self.cyst_tensors=[]

        for filename in os.listdir(self.normal_path):
            img=cv2.imread(os.path.join(self.normal_path, filename),1)
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img, (224, 224))
            self.normal_tensors.append(img)
            self.normal_labels.append([1, 0, 0, 0])

        for filename in os.listdir(self.tumor_path):
            img=cv2.imread(os.path.join(self.tumor_path, filename),1)
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img, (224, 224))
            self.tumor_tensors.append(img)
            self.tumor_labels.append([0, 1, 0, 0])

        for filename in os.listdir(self.stone_path):
            img=cv2.imread(os.path.join(self.stone_path, filename),1)
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img, (224, 224))
            self.stone_tensors.append(img)
            self.stone_labels.append([0, 0, 1, 0])

        for filename in os.listdir(self.cyst_path):
            img=cv2.imread(os.path.join(self.cyst_path, filename),1)
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img, (224, 224))
            self.cyst_tensors.append(img)
            self.cyst_labels.append([0, 0, 0, 1])

        self.normal_tensors=np.array(self.normal_tensors, dtype=np.uint8)
        self.tumor_tensors=np.array(self.tumor_tensors, dtype=np.uint8)
        self.stone_tensors=np.array(self.stone_tensors, dtype=np.uint8)
        self.cyst_tensors=np.array(self.cyst_tensors, dtype=np.uint8)

        rng = np.random.default_rng(42)
        self.x=np.concatenate((self.normal_tensors, self.tumor_tensors, self.stone_tensors, self.cyst_tensors), axis=0)
        rng.shuffle(self.x)

        del self.normal_tensors
        del self.tumor_tensors
        del self.stone_tensors
        del self.cyst_tensors

        self.normal_labels=np.array(self.normal_labels, dtype=np.uint8)
        self.tumor_labels=np.array(self.tumor_labels, dtype=np.uint8)
        self.stone_labels=np.array(self.stone_labels, dtype=np.uint8)
        self.cyst_labels=np.array(self.cyst_labels, dtype=np.uint8)

        self.y=np.concatenate((self.normal_labels, self.tumor_labels, self.stone_labels, self.cyst_labels), dtype=np.int8)
        rng.shuffle(self.y)

        del self.normal_labels
        del self.tumor_labels
        del self.stone_labels
        del self.cyst_labels

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.model.compile(optimizer='adam', loss='categorical_crossentropy')
        self.model.summary()
        self.model.fit(self.x, self.y, epochs=self.config.params_epochs)
        self.save_model(self.config.trained_model_path, self.model)

In [11]:
training_config_generator=ConfigurationManager()
training_config=training_config_generator.get_training_configs()

[2024-08-17 01:15:24,053: INFO: common: yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Kidney disease detection\config\config.yaml loaded successfully]
[2024-08-17 01:15:24,054: INFO: common: yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Kidney disease detection\params.yaml loaded successfully]


In [12]:
trainer=Trainer(training_config)
trainer.get_base_model()
#trainer.load_data()

[2024-08-17 01:15:43,770: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [9]:
trainer.train()

In [13]:
trainer.model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       100,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,815,046 (56.51 MB)

 Trainable params: 100,356 (392.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)